In [1]:
#https://blog.socratesk.com/blog/2018/01/29/expose-ML-model-as-REST-API
import torch
from torch import nn, optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models

In [10]:
num_workers = 0
batch_size = 64
transform = transforms.Compose([
    transforms.Resize(300),
    transforms.CenterCrop(300),
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])

tv_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])

train_data = datasets.ImageFolder('chest_xray/train', transform=transform)
valid_data = datasets.ImageFolder('chest_xray/val', transform=tv_transform)
test_data = datasets.ImageFolder('chest_xray/test', transform=tv_transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

loaders = [train_loader, valid_loader, test_loader]

FileNotFoundError: [Errno 2] No such file or directory: 'chest_xray/train'

In [11]:
transfer = models.alexnet(pretrained=True)
transfer.classifier[6] = nn.Linear(4096, 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transfer.parameters(), lr=0.0001)

In [12]:
def train(n_epochs, loader, model, optimizer, criterion, use_cuda, save_path):
    valid_loss_min = .091
    train_loss = 0
    valid_loss = 0
    for epoch in range(1, n_epochs+1):
        model.train()
        for batch_idx, (data, target) in enumerate(loader[0]):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            #if batch_idx == 0:
                #print('Started Batch')
            optimizer.zero_grad()       
            output = model(data)        
            loss = criterion(output, target)        
            loss.backward()        
            optimizer.step() 
            torch.cuda.empty_cache() 
            train_loss += loss.item()*data.size(0)
            #if batch_idx % show == 0:
                #print('Training: ' + str(batch_idx))
        model.eval()
        for batch_idx, (data, target) in enumerate(loader[1]):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            torch.cuda.empty_cache()
            #if batch_idx % show == 0:
                #print('Validation: ' + str(batch_idx))
        train_loss = train_loss/len(loader[0].dataset)
        valid_loss = valid_loss/len(loader[1].dataset)
        
        print('Epoch: {} \tTraining Loss: {:.3f} \tValidation Loss: {:.3f}'.format(
            epoch, train_loss, valid_loss))
        
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.3f} --> {:.3f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    return model

In [13]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
        
    model.eval()
    for batch_idx, (data, target) in enumerate(loaders[2]):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [16]:
transfer.load_state_dict(torch.load('model.pth'))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location='cpu' to map your storages to the CPU.

In [26]:
test(loaders, transfer, criterion, True)

Test Loss: 0.914777


Test Accuracy: 83% (524/624)


In [32]:
import pickle
File = open('Model.pckl', 'wb')
pickle.dump(transfer, File)
File.close()